<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/Alpha_1%2B1_orderbook_indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model incorporates both - indicators like RSI OBV ATR and also order book buying selling pressure and liquidity

In [ ]:
!pip install ccxt pandas numpy scikit-learn tensorflow pandas-ta matplotlib plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 10.0 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=fb1c4b091771392fc95c75932ed2e5dd315339f07743e6d359f31e376c54c8ae
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.12
    Uninstalling aiohttp-3.11.12:
      Successfully uninstalled aiohttp-3.11.12


some modifications
active and passive liquidity from aggressive buy/sell market orders and limit orders as passive liquidity
order book depth as std instead of 100 bid/asks

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
import logging
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize KuCoin Exchange
exchange = ccxt.kucoin()

# Fetch Historical Data (OHLCV)
def fetch_historical_data(symbol, timeframe, limit=500):
    """Fetch OHLCV data from KuCoin."""
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df
    except Exception as e:
        logging.error(f"Error fetching historical data for {symbol}: {e}")
        return pd.DataFrame()

# Add Technical Indicators
def add_technical_indicators(df):
    """Add RSI, ATR, OBV, SMA, and EMA to the DataFrame."""
    df['RSI'] = df.ta.rsi(close='close', length=14)
    df['ATR'] = df.ta.atr(high='high', low='low', close='close', length=14)
    df['OBV'] = df.ta.obv(close='close', volume='volume')
    df['SMA_50'] = df.ta.sma(close='close', length=50)
    df['EMA_50'] = df.ta.ema(close='close', length=50)
    return df

# Fetch Level 2 Order Book
def fetch_order_book(symbol, depth=100):
    """Fetch bid-ask Level 2 order book data from KuCoin."""
    try:
        order_book = exchange.fetch_order_book(symbol, limit=depth)
        bids = order_book['bids'][:depth]  # Top bids
        asks = order_book['asks'][:depth]  # Top asks
        return bids, asks
    except Exception as e:
        logging.error(f"Error fetching order book for {symbol}: {e}")
        return [], []

# Fetch Executed Trade Data
def fetch_trade_data(symbol, since=None, limit=1000):
    """Fetch executed market orders from KuCoin."""
    try:
        trades = exchange.fetch_trades(symbol, since=since, limit=limit)
        df = pd.DataFrame(trades, columns=['timestamp', 'price', 'amount', 'side'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        logging.error(f"Error fetching trade data for {symbol}: {e}")
        return pd.DataFrame()

# Classify Active and Passive Liquidity
def classify_liquidity(trade_df, bids, asks, tolerance=0.001):
    """Classify trades as active liquidity (market orders) and passive liquidity (limit orders)."""
    if trade_df.empty or not bids or not asks:
        return trade_df

    best_bid = bids[0][0] if bids else None
    best_ask = asks[0][0] if asks else None

    if not best_bid or not best_ask:
        return trade_df

    # Classify active liquidity (market orders)
    trade_df['active_side'] = trade_df.apply(
        lambda row: 'buy' if abs(row['price'] - best_ask) <= best_ask * tolerance else (
            'sell' if abs(row['price'] - best_bid) <= best_bid * tolerance else None
        ),
        axis=1
    )

    # Weight active liquidity by trade size
    trade_df['weighted_amount'] = trade_df['amount'] * trade_df['price']
    return trade_df

# Analyze Active and Passive Liquidity
def analyze_liquidity(trade_df, timeframe):
    """Analyze active and passive liquidity over a given timeframe."""
    try:
        if trade_df.empty:
            raise ValueError("Trade data is empty.")

        trade_df = trade_df.set_index('timestamp')
        resampled = trade_df.resample(timeframe).agg({
            'weighted_amount': 'sum',  # Weighted by trade size
            'active_side': lambda x: (x == 'buy').sum()
        })

        resampled.rename(columns={'active_side': 'active_buy_count'}, inplace=True)
        resampled['active_sell_count'] = len(trade_df) - resampled['active_buy_count']

        resampled['active_buy_volume'] = trade_df[trade_df['active_side'] == 'buy'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()
        resampled['active_sell_volume'] = trade_df[trade_df['active_side'] == 'sell'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()

        resampled['order_flow_imbalance'] = resampled['active_buy_volume'] - resampled['active_sell_volume']
        resampled['cumulative_order_flow'] = resampled['order_flow_imbalance'].cumsum()  # Cumulative Volume Delta (CVD)

        return resampled
    except Exception as e:
        logging.error(f"Error analyzing liquidity for {timeframe}: {e}")
        return pd.DataFrame()

# Calculate Order Book Depth with Standard Deviation
def calculate_order_book_depth_with_std(bids, asks, current_price, std_dev):
    """Calculate order book depth within ±1 standard deviation of the current price."""
    try:
        price_range_low = current_price - std_dev
        price_range_high = current_price + std_dev

        # Filter bids and asks within the price range
        bid_volumes = [bid[1] for bid in bids if price_range_low <= bid[0] <= price_range_high]
        ask_volumes = [ask[1] for ask in asks if price_range_low <= ask[0] <= price_range_high]

        return sum(bid_volumes), sum(ask_volumes)
    except Exception as e:
        logging.error(f"Error calculating order book depth with std: {e}")
        return 0, 0

# Identify High-Quality Liquidity Zones
def identify_liquidity_zones(bids, asks, volume_threshold=100, price_tolerance=0.001):
    """Identify high-quality liquidity zones (areas with significantly higher liquidity)."""
    try:
        # Extract prices and volumes
        bid_prices = [bid[0] for bid in bids]
        bid_volumes = [bid[1] for bid in bids]
        ask_prices = [ask[0] for ask in asks]
        ask_volumes = [ask[1] for ask in asks]

        # Function to group prices into zones
        def group_prices(prices, volumes, tolerance):
            zones = []
            current_zone = []
            current_price = None

            for price, volume in zip(prices, volumes):
                if current_price is None or abs(price - current_price) <= tolerance:
                    current_zone.append((price, volume))
                    current_price = price
                else:
                    zones.append(current_zone)
                    current_zone = [(price, volume)]
                    current_price = price

            if current_zone:
                zones.append(current_zone)
            return zones

        # Group bid and ask prices into zones
        bid_zones = group_prices(bid_prices, bid_volumes, price_tolerance)
        ask_zones = group_prices(ask_prices, ask_volumes, price_tolerance)

        # Filter zones based on volume threshold
        def filter_zones(zones, threshold):
            filtered = []
            for zone in zones:
                total_volume = sum(vol for _, vol in zone)
                if total_volume >= threshold:
                    avg_price = sum(price * vol for price, vol in zone) / total_volume
                    filtered.append((avg_price, total_volume))
            return filtered

        bid_liquidity_zones = filter_zones(bid_zones, volume_threshold)
        ask_liquidity_zones = filter_zones(ask_zones, volume_threshold)

        return bid_liquidity_zones, ask_liquidity_zones
    except Exception as e:
        logging.error(f"Error identifying liquidity zones: {e}")
        return [], []

# Preprocess Data for LSTM
def preprocess_data(df, sequence_length=50):
    """Preprocess data for LSTM model."""
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM Model
def build_model(input_shape):
    """Build and compile the LSTM model."""
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict Price
def predict_price(symbol, sequence_length=50):
    """Predict price using LSTM model."""
    df_15m = fetch_historical_data(symbol, '15m', limit=500)
    df_4h = fetch_historical_data(symbol, '4h', limit=500)

    df_15m = add_technical_indicators(df_15m)
    df_4h = add_technical_indicators(df_4h)

    X_15m, _, scaler_15m = preprocess_data(df_15m, sequence_length)
    X_4h, _, scaler_4h = preprocess_data(df_4h, sequence_length)

    model_15m = build_model((X_15m.shape[1], X_15m.shape[2]))
    model_4h = build_model((X_4h.shape[1], X_4h.shape[2]))

    model_15m.fit(X_15m, X_15m[:, -1, 0], epochs=5, batch_size=32, verbose=1)
    model_4h.fit(X_4h, X_4h[:, -1, 0], epochs=5, batch_size=32, verbose=1)

    latest_sequence_15m = X_15m[-1].reshape(1, X_15m.shape[1], X_15m.shape[2])
    latest_sequence_4h = X_4h[-1].reshape(1, X_4h.shape[1], X_4h.shape[2])

    predicted_price_15m = scaler_15m.inverse_transform([[model_15m.predict(latest_sequence_15m)[0][0], 0, 0, 0, 0, 0]])[0][0]
    predicted_price_4h = scaler_4h.inverse_transform([[model_4h.predict(latest_sequence_4h)[0][0], 0, 0, 0, 0, 0]])[0][0]

    return predicted_price_15m, predicted_price_4h

# Main Function
def analyze_asset(symbol):
    """Analyze an asset using order flow, technical indicators, and LSTM predictions."""
    try:
        # Fetch data
        bids, asks = fetch_order_book(symbol, depth=100)
        trade_data = fetch_trade_data(symbol)
        df_15m = fetch_historical_data(symbol, '15m', limit=500)
        df_4h = fetch_historical_data(symbol, '4h', limit=500)

        if trade_data.empty or df_15m.empty or df_4h.empty:
            logging.warning(f"No data found for {symbol}. Skipping...")
            return

        # Add technical indicators
        df_15m = add_technical_indicators(df_15m)
        df_4h = add_technical_indicators(df_4h)

        # Classify active and passive liquidity
        trade_data = classify_liquidity(trade_data, bids, asks)

        # Analyze active and passive liquidity
        liquidity_4h = analyze_liquidity(trade_data, '4h')
        liquidity_15m = analyze_liquidity(trade_data, '15min')

        # Calculate order book depth with 1 standard deviation (1σ)
        current_price = df_15m['close'].iloc[-1]
        std_dev = df_15m['close'].std()  # Standard deviation of 1-hour prices
        bid_depth, ask_depth = calculate_order_book_depth_with_std(bids, asks, current_price, std_dev)

        # Identify liquidity zones
        bid_liquidity_zones, ask_liquidity_zones = identify_liquidity_zones(bids, asks, volume_threshold=100, price_tolerance=0.001)

        # Predict prices
        predicted_price_15m, predicted_price_4h = predict_price(symbol)

        # Generate trading signal
        if predicted_price_15m > current_price * 1.002:  # 0.2% threshold
            signal = "🚀 BUY Signal"
        elif predicted_price_15m < current_price * 0.998:  # 0.2% threshold
            signal = "🔻 SELL Signal"
        else:
            signal = "❌ NO TRADE"

        # Output results
        print(f"\n🔹 {symbol}")
        print(f"📈 Current Price: {current_price:.2f}")
        print(f"📊 Trading Signal: {signal}")
        print(f"📊 15M Prediction: {predicted_price_15m:.2f}")
        print(f"📊 4H Prediction: {predicted_price_4h:.2f}")
        print(f"📊 4H Order Flow Imbalance: {liquidity_4h['order_flow_imbalance'].iloc[-1]:.2f}")
        print(f"📊 15M Order Flow Imbalance: {liquidity_15m['order_flow_imbalance'].iloc[-1]:.2f}")
        print(f"📊 4H Active Buy Volume: {liquidity_4h['active_buy_volume'].iloc[-1]:.2f}")
        print(f"📊 4H Active Sell Volume: {liquidity_4h['active_sell_volume'].iloc[-1]:.2f}")
        print(f"📊 15M Active Buy Volume: {liquidity_15m['active_buy_volume'].iloc[-1]:.2f}")
        print(f"📊 15M Active Sell Volume: {liquidity_15m['active_sell_volume'].iloc[-1]:.2f}")
        print(f"📊 Order Book Depth (1σ): {bid_depth:.2f} / {ask_depth:.2f}")
        print(f"📊 Bid Liquidity Zones: {[f'{price:.4f} ({vol:.2f})' for price, vol in bid_liquidity_zones]}")
        print(f"📊 Ask Liquidity Zones: {[f'{price:.4f} ({vol:.2f})' for price, vol in ask_liquidity_zones]}")
        print(f"📊 RSI: {df_15m['RSI'].iloc[-1]:.2f}")
        print(f"📊 ATR: {df_15m['ATR'].iloc[-1]:.2f}")
        print(f"📊 OBV: {df_15m['OBV'].iloc[-1]:.2f}")
    except Exception as e:
        logging.error(f"\n🔹 {symbol}")
        logging.error(f"⚠️ Error: {e}")

# List of Assets to Analyze
assets = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT', 'FET/USDT', 'NEAR/USDT', 'AVAX/USDT', 'DOGE/USDT',
          'XRP/USDT', 'LINK/USDT', 'INJ/USDT', 'HNT/USDT', 'CRV/USDT', 'SOL/USDT', 'LTC/USDT',
          'MNT/USDT', 'AAVE/USDT', 'JUP/USDT']

# Analyze each asset
for asset in assets:
    analyze_asset(asset)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 0.0469
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0145
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0074
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0049
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0042
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 0.1494
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0276
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0181
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0148
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0103
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step

🔹 BTC/USDT
📈 Current Price: 96975.50
📊 Trading Signal: ❌ NO TRADE
📊 15M Prediction: 97168.02
📊 4H Prediction: 96867.77
📊 4H Order Flow Imbalance: nan
📊 15M Order Flow Imbalance: nan
📊 4H Active Buy Volume: 1842.35
📊 4H Active Sell Volume: nan
📊 15M Active Buy Volume: 1842.35
📊 15M

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - loss: 0.0792
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0160
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0105
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0072
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0045
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0985
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0166
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0062
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0056
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0044
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step

🔹 ETH/USDT
📈 Current Price: 2661.86
📊 Trading Signal: ❌ NO TRADE
📊 15M Prediction: 2664.36
📊 4H Prediction: 2684.50
📊 4H Order Flow Imbalance: 17755.76
📊 15M Order Flow Imbalance: 17755.76
📊 4H Active Buy Volume: 17862.16
📊 4H Active Sell Volume: 106.39
📊 15M Active Buy Volume: 17

...

13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - loss: 0.1554
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.0198
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0100
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0071
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0041
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - loss: 0.1077
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0142
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0091
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0070
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.0055
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 509ms/step

🔹 MNT/USDT
📈 Current Price: 1.05
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 1.05
📊 4H Prediction: 1.08
📊 4H Order Flow Imbalance: -491.62
📊 15M Order Flow Imbalance: nan
📊 4H Active Buy Volume: 51.02
📊 4H Active Sell Volume: 542.64
📊 15M Active Buy Volume: nan
📊 15M Active S

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - loss: 0.1006
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0124
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0045
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0028
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0024
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.3257
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0261
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0172
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0139
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0119
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step

🔹 AAVE/USDT
📈 Current Price: 245.86
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 247.11
📊 4H Prediction: 278.51
📊 4H Order Flow Imbalance: 16339.40
📊 15M Order Flow Imbalance: 16339.40
📊 4H Active Buy Volume: 17082.38
📊 4H Active Sell Volume: 742.98
📊 15M Active Buy Volume: 17

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.0712
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0082
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0036
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0023
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.0020
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0967
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0256
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0146
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0122
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0091
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step

🔹 JUP/USDT
📈 Current Price: 0.85
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 0.86
📊 4H Prediction: 0.81
📊 4H Order Flow Imbalance: 1317.87
📊 15M Order Flow Imbalance: -93.85
📊 4H Active Buy Volume: 1690.24
📊 4H Active Sell Volume: 372.37
📊 15M Active Buy Volume: 11.13
📊 15M A

order book imbalance passive liquidity with limit orders in queue. we already have active liqudity or aggressive buy/sell orders already in.

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
import logging
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize KuCoin Exchange
exchange = ccxt.kucoin()

# Fetch Historical Data (OHLCV)
def fetch_historical_data(symbol, timeframe, limit=500):
    """Fetch OHLCV data from KuCoin."""
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df
    except Exception as e:
        logging.error(f"Error fetching historical data for {symbol}: {e}")
        return pd.DataFrame()

# Add Technical Indicators
def add_technical_indicators(df):
    """Add RSI, ATR, OBV, SMA, and EMA to the DataFrame."""
    df['RSI'] = df.ta.rsi(close='close', length=14)
    df['ATR'] = df.ta.atr(high='high', low='low', close='close', length=14)
    df['OBV'] = df.ta.obv(close='close', volume='volume')
    df['SMA_50'] = df.ta.sma(close='close', length=50)
    df['EMA_50'] = df.ta.ema(close='close', length=50)
    return df

# Fetch Level 2 Order Book
def fetch_order_book(symbol, depth=100):
    """Fetch bid-ask Level 2 order book data from KuCoin."""
    try:
        order_book = exchange.fetch_order_book(symbol, limit=depth)
        bids = order_book['bids'][:depth]  # Top bids
        asks = order_book['asks'][:depth]  # Top asks
        return bids, asks
    except Exception as e:
        logging.error(f"Error fetching order book for {symbol}: {e}")
        return [], []

# Fetch Executed Trade Data
def fetch_trade_data(symbol, since=None, limit=1000):
    """Fetch executed market orders from KuCoin."""
    try:
        trades = exchange.fetch_trades(symbol, since=since, limit=limit)
        df = pd.DataFrame(trades, columns=['timestamp', 'price', 'amount', 'side'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        logging.error(f"Error fetching trade data for {symbol}: {e}")
        return pd.DataFrame()

# Classify Active and Passive Liquidity
def classify_liquidity(trade_df, bids, asks, tolerance=0.001):
    """Classify trades as active liquidity (market orders) and passive liquidity (limit orders)."""
    if trade_df.empty or not bids or not asks:
        return trade_df

    best_bid = bids[0][0] if bids else None
    best_ask = asks[0][0] if asks else None

    if not best_bid or not best_ask:
        return trade_df

    # Classify active liquidity (market orders)
    trade_df['active_side'] = trade_df.apply(
        lambda row: 'buy' if abs(row['price'] - best_ask) <= best_ask * tolerance else (
            'sell' if abs(row['price'] - best_bid) <= best_bid * tolerance else None
        ),
        axis=1
    )

    # Weight active liquidity by trade size
    trade_df['weighted_amount'] = trade_df['amount'] * trade_df['price']
    return trade_df

# Analyze Active and Passive Liquidity
def analyze_liquidity(trade_df, timeframe):
    """Analyze active and passive liquidity over a given timeframe."""
    try:
        if trade_df.empty:
            raise ValueError("Trade data is empty.")

        trade_df = trade_df.set_index('timestamp')
        resampled = trade_df.resample(timeframe).agg({
            'weighted_amount': 'sum',  # Weighted by trade size
            'active_side': lambda x: (x == 'buy').sum()
        })

        resampled.rename(columns={'active_side': 'active_buy_count'}, inplace=True)
        resampled['active_sell_count'] = len(trade_df) - resampled['active_buy_count']

        resampled['active_buy_volume'] = trade_df[trade_df['active_side'] == 'buy'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()
        resampled['active_sell_volume'] = trade_df[trade_df['active_side'] == 'sell'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()

        resampled['order_flow_imbalance'] = resampled['active_buy_volume'] - resampled['active_sell_volume']
        resampled['cumulative_order_flow'] = resampled['order_flow_imbalance'].cumsum()  # Cumulative Volume Delta (CVD)

        return resampled
    except Exception as e:
        logging.error(f"Error analyzing liquidity for {timeframe}: {e}")
        return pd.DataFrame()

# Calculate Passive Liquidity (Limit Orders)
def calculate_passive_liquidity(bids, asks, current_price, std_dev):
    """Calculate passive liquidity (limit orders) within ±1 standard deviation."""
    price_range_low = current_price - std_dev
    price_range_high = current_price + std_dev

    # Filter bids and asks within the price range
    bid_volumes = [bid[1] for bid in bids if price_range_low <= bid[0] <= price_range_high]
    ask_volumes = [ask[1] for ask in asks if price_range_low <= ask[0] <= price_range_high]

    return sum(bid_volumes), sum(ask_volumes)

# Calculate Order Book Imbalance
def calculate_order_book_imbalance(bids, asks, current_price, std_dev):
    """Calculate order book imbalance within ±1 standard deviation."""
    price_range_low = current_price - std_dev
    price_range_high = current_price + std_dev

    # Filter bids and asks within the price range
    bid_volumes = [bid[1] for bid in bids if price_range_low <= bid[0] <= price_range_high]
    ask_volumes = [ask[1] for ask in asks if price_range_low <= ask[0] <= price_range_high]

    total_bid_volume = sum(bid_volumes)
    total_ask_volume = sum(ask_volumes)

    if total_bid_volume + total_ask_volume == 0:
        return 0  # Avoid division by zero

    order_book_imbalance = (total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)
    return order_book_imbalance

# Preprocess Data for LSTM
def preprocess_data(df, sequence_length=50):
    """Preprocess data for LSTM model."""
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM Model
def build_model(input_shape):
    """Build and compile the LSTM model."""
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict Price
def predict_price(symbol, sequence_length=50):
    """Predict price using LSTM model."""
    df_15m = fetch_historical_data(symbol, '15m', limit=500)
    df_4h = fetch_historical_data(symbol, '4h', limit=500)

    df_15m = add_technical_indicators(df_15m)
    df_4h = add_technical_indicators(df_4h)

    X_15m, _, scaler_15m = preprocess_data(df_15m, sequence_length)
    X_4h, _, scaler_4h = preprocess_data(df_4h, sequence_length)

    model_15m = build_model((X_15m.shape[1], X_15m.shape[2]))
    model_4h = build_model((X_4h.shape[1], X_4h.shape[2]))

    model_15m.fit(X_15m, X_15m[:, -1, 0], epochs=5, batch_size=32, verbose=1)
    model_4h.fit(X_4h, X_4h[:, -1, 0], epochs=5, batch_size=32, verbose=1)

    latest_sequence_15m = X_15m[-1].reshape(1, X_15m.shape[1], X_15m.shape[2])
    latest_sequence_4h = X_4h[-1].reshape(1, X_4h.shape[1], X_4h.shape[2])

    predicted_price_15m = scaler_15m.inverse_transform([[model_15m.predict(latest_sequence_15m)[0][0], 0, 0, 0, 0, 0]])[0][0]
    predicted_price_4h = scaler_4h.inverse_transform([[model_4h.predict(latest_sequence_4h)[0][0], 0, 0, 0, 0, 0]])[0][0]

    return predicted_price_15m, predicted_price_4h

# Main Function
def analyze_asset(symbol):
    """Analyze an asset using order flow, technical indicators, and LSTM predictions."""
    try:
        # Fetch data
        bids, asks = fetch_order_book(symbol, depth=100)
        trade_data = fetch_trade_data(symbol)
        df_15m = fetch_historical_data(symbol, '15m', limit=500)
        df_4h = fetch_historical_data(symbol, '4h', limit=500)

        if trade_data.empty or df_15m.empty or df_4h.empty:
            logging.warning(f"No data found for {symbol}. Skipping...")
            return

        # Add technical indicators
        df_15m = add_technical_indicators(df_15m)
        df_4h = add_technical_indicators(df_4h)

        # Classify active and passive liquidity
        trade_data = classify_liquidity(trade_data, bids, asks)

        # Analyze active and passive liquidity
        liquidity_4h = analyze_liquidity(trade_data, '4h')
        liquidity_15m = analyze_liquidity(trade_data, '15min')

        # Calculate dynamic standard deviation
        std_dev = df_15m['close'].std()  # Standard deviation of 1-hour prices
        current_price = df_15m['close'].iloc[-1]

        # Calculate passive liquidity
        passive_bid_liquidity, passive_ask_liquidity = calculate_passive_liquidity(bids, asks, current_price, std_dev)

        # Calculate order book imbalance
        order_book_imbalance = calculate_order_book_imbalance(bids, asks, current_price, std_dev)

        # Predict prices
        predicted_price_15m, predicted_price_4h = predict_price(symbol)

        # Generate trading signal
        if predicted_price_15m > current_price * 1.002:  # 0.2% threshold
            signal = "🚀 BUY Signal"
        elif predicted_price_15m < current_price * 0.998:  # 0.2% threshold
            signal = "🔻 SELL Signal"
        else:
            signal = "❌ NO TRADE"

        # Output results
        print(f"\n🔹 {symbol}")
        print(f"📈 Current Price: {current_price:.2f}")
        print(f"📊 Trading Signal: {signal}")
        print(f"📊 15M Prediction: {predicted_price_15m:.2f}")
        print(f"📊 4H Prediction: {predicted_price_4h:.2f}")
        print(f"📊 4H Order Flow Imbalance: {liquidity_4h['order_flow_imbalance'].iloc[-1]:.2f}")
        print(f"📊 15M Order Flow Imbalance: {liquidity_15m['order_flow_imbalance'].iloc[-1]:.2f}")
        print(f"📊 Passive Bid Liquidity: {passive_bid_liquidity:.2f}")
        print(f"📊 Passive Ask Liquidity: {passive_ask_liquidity:.2f}")
        print(f"📊 Order Book Imbalance: {order_book_imbalance:.2f}")
        print(f"📊 RSI: {df_15m['RSI'].iloc[-1]:.2f}")
        print(f"📊 ATR: {df_15m['ATR'].iloc[-1]:.2f}")
        print(f"📊 OBV: {df_15m['OBV'].iloc[-1]:.2f}")
    except Exception as e:
        logging.error(f"\n🔹 {symbol}")
        logging.error(f"⚠️ Error: {e}")

# List of Assets to Analyze
assets = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT', 'FET/USDT', 'NEAR/USDT', 'AVAX/USDT', 'DOGE/USDT',
          'XRP/USDT', 'LINK/USDT', 'INJ/USDT', 'HNT/USDT', 'CRV/USDT', 'SOL/USDT', 'LTC/USDT',
          'MNT/USDT', 'AAVE/USDT', 'JUP/USDT']

# Analyze each asset
for asset in assets:
    analyze_asset(asset)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 160ms/step - loss: 0.0820
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - loss: 0.0156
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 0.0070
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0060
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0051
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - loss: 0.0638
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0240
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - loss: 0.0149
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0104
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0090
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step

🔹 BTC/USDT
📈 Current Price: 96302.30
📊 Trading Signal: ❌ NO TRADE
📊 15M Prediction: 96253.47
📊 4H Prediction: 96746.02
📊 4H Order Flow Imbalance: nan
📊 15M Order Flow Imbalance: nan
📊 Passive Bid Liquidity: 15.74
📊 Passive Ask Liquidity: 10.05
📊 Order Book Imbalance: 0.22
📊 RSI: 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.1061
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0161
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0075
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0040
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0036
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 0.1315
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0147
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0090
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0066
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0054
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step

🔹 ETH/USDT
📈 Current Price: 2645.74
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 2656.86
📊 4H Prediction: 2687.51
📊 4H Order Flow Imbalance: nan
📊 15M Order Flow Imbalance: nan
📊 Passive Bid Liquidity: 261.95
📊 Passive Ask Liquidity: 399.63
📊 Order Book Imbalance: -0.21
📊 RSI:

...

13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - loss: 0.1569
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0161
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0104
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0055
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0045
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.4059
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0317
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 0.0165
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0124
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0103
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 621ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step

🔹 MNT/USDT
📈 Current Price: 1.03
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 1.04
📊 4H Prediction: 1.11
📊 4H Order Flow Imbalance: nan
📊 15M Order Flow Imbalance: nan
📊 Passive Bid Liquidity: 214689.75
📊 Passive Ask Liquidity: 220095.86
📊 Order Book Imbalance: -0.01
📊 RSI: 36

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.0502
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0100
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0054
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0026
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0027
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.1599
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0213
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0139
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0106
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0080
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step

🔹 AAVE/USDT
📈 Current Price: 241.23
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 243.82
📊 4H Prediction: 267.23
📊 4H Order Flow Imbalance: nan
📊 15M Order Flow Imbalance: nan
📊 Passive Bid Liquidity: 379.03
📊 Passive Ask Liquidity: 468.04
📊 Order Book Imbalance: -0.11
📊 RSI: 3

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 0.0631
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0118
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0046
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0027
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0021
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 0.0860
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0196
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0142
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0094
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0082
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step

🔹 JUP/USDT
📈 Current Price: 0.82
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 0.83
📊 4H Prediction: 0.83
📊 4H Order Flow Imbalance: nan
📊 15M Order Flow Imbalance: nan
📊 Passive Bid Liquidity: 204959.68
📊 Passive Ask Liquidity: 171683.56
📊 Order Book Imbalance: 0.09
📊 RSI: 28.

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
import logging
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize KuCoin Exchange
exchange = ccxt.kucoin()

# Fetch Historical Data (OHLCV)
def fetch_historical_data(symbol, timeframe, limit=500):
    """Fetch OHLCV data from KuCoin."""
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df
    except Exception as e:
        logging.error(f"Error fetching historical data for {symbol}: {e}")
        return pd.DataFrame()

# Add Technical Indicators
def add_technical_indicators(df):
    """Add RSI, ATR, OBV, SMA, and EMA to the DataFrame."""
    df['RSI'] = df.ta.rsi(close='close', length=14)
    df['ATR'] = df.ta.atr(high='high', low='low', close='close', length=14)
    df['OBV'] = df.ta.obv(close='close', volume='volume')
    df['SMA_50'] = df.ta.sma(close='close', length=50)
    df['EMA_50'] = df.ta.ema(close='close', length=50)
    return df

# Fetch Level 2 Order Book
def fetch_order_book(symbol, depth=100):
    """Fetch bid-ask Level 2 order book data from KuCoin."""
    try:
        order_book = exchange.fetch_order_book(symbol, limit=depth)
        bids = order_book['bids'][:depth]  # Top bids
        asks = order_book['asks'][:depth]  # Top asks
        return bids, asks
    except Exception as e:
        logging.error(f"Error fetching order book for {symbol}: {e}")
        return [], []

# Fetch Executed Trade Data
def fetch_trade_data(symbol, since=None, limit=1000):
    """Fetch executed market orders from KuCoin."""
    try:
        trades = exchange.fetch_trades(symbol, since=since, limit=limit)
        df = pd.DataFrame(trades, columns=['timestamp', 'price', 'amount', 'side'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        logging.error(f"Error fetching trade data for {symbol}: {e}")
        return pd.DataFrame()

# Classify Aggressive Orders with Trade Size Weighting
def classify_aggressive_orders(trade_df, bids, asks, tolerance=0.001):
    """Classify trades as aggressive buys or sells based on a tolerance threshold and weight by trade size."""
    if trade_df.empty or not bids or not asks:
        return trade_df

    best_bid = bids[0][0] if bids else None
    best_ask = asks[0][0] if asks else None

    if not best_bid or not best_ask:
        return trade_df

    # Classify aggressive orders
    trade_df['aggressive_side'] = trade_df.apply(
        lambda row: 'buy' if abs(row['price'] - best_ask) <= best_ask * tolerance else (
            'sell' if abs(row['price'] - best_bid) <= best_bid * tolerance else None
        ),
        axis=1
    )

    # Weight aggressive orders by trade size
    trade_df['weighted_amount'] = trade_df['amount'] * trade_df['price']
    return trade_df

# Analyze Aggressive Buying & Selling
def analyze_aggressive_orders(trade_df, timeframe):
    """Analyze aggressive buying and selling over a given timeframe."""
    try:
        if trade_df.empty:
            raise ValueError("Trade data is empty.")

        trade_df = trade_df.set_index('timestamp')
        resampled = trade_df.resample(timeframe).agg({
            'weighted_amount': 'sum',  # Weighted by trade size
            'aggressive_side': lambda x: (x == 'buy').sum()
        })

        resampled.rename(columns={'aggressive_side': 'aggressive_buy_count'}, inplace=True)
        resampled['aggressive_sell_count'] = len(trade_df) - resampled['aggressive_buy_count']

        resampled['aggressive_buy_volume'] = trade_df[trade_df['aggressive_side'] == 'buy'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()
        resampled['aggressive_sell_volume'] = trade_df[trade_df['aggressive_side'] == 'sell'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()

        resampled['delta'] = resampled['aggressive_buy_volume'] - resampled['aggressive_sell_volume']
        resampled['cumulative_delta'] = resampled['delta'].cumsum()  # CVD

        return resampled
    except Exception as e:
        logging.error(f"Error analyzing aggressive orders for {timeframe}: {e}")
        return pd.DataFrame()

# Calculate Order Book Depth (Top 100 Bids/Asks)
def calculate_order_book_depth(bids, asks, depth=100):
    """Calculate total volume at different price levels (e.g., top 100 bids/asks)."""
    try:
        bid_volumes = [bid[1] for bid in bids[:depth]]
        ask_volumes = [ask[1] for ask in asks[:depth]]
        return sum(bid_volumes), sum(ask_volumes)
    except Exception as e:
        logging.error(f"Error calculating order book depth: {e}")
        return 0, 0

# Identify High-Quality Liquidity Zones
def identify_liquidity_zones(bids, asks, volume_threshold=100, price_tolerance=0.001):
    """Identify high-quality liquidity zones (areas with significantly higher liquidity)."""
    try:
        # Extract prices and volumes
        bid_prices = [bid[0] for bid in bids]
        bid_volumes = [bid[1] for bid in bids]
        ask_prices = [ask[0] for ask in asks]
        ask_volumes = [ask[1] for ask in asks]

        # Function to group prices into zones
        def group_prices(prices, volumes, tolerance):
            zones = []
            current_zone = []
            current_price = None

            for price, volume in zip(prices, volumes):
                if current_price is None or abs(price - current_price) <= tolerance:
                    current_zone.append((price, volume))
                    current_price = price
                else:
                    zones.append(current_zone)
                    current_zone = [(price, volume)]
                    current_price = price

            if current_zone:
                zones.append(current_zone)
            return zones

        # Group bid and ask prices into zones
        bid_zones = group_prices(bid_prices, bid_volumes, price_tolerance)
        ask_zones = group_prices(ask_prices, ask_volumes, price_tolerance)

        # Filter zones based on volume threshold
        def filter_zones(zones, threshold):
            filtered = []
            for zone in zones:
                total_volume = sum(vol for _, vol in zone)
                if total_volume >= threshold:
                    avg_price = sum(price * vol for price, vol in zone) / total_volume
                    filtered.append((avg_price, total_volume))
            return filtered

        bid_liquidity_zones = filter_zones(bid_zones, volume_threshold)
        ask_liquidity_zones = filter_zones(ask_zones, volume_threshold)

        return bid_liquidity_zones, ask_liquidity_zones
    except Exception as e:
        logging.error(f"Error identifying liquidity zones: {e}")
        return [], []

# Preprocess Data for LSTM
def preprocess_data(df, sequence_length=50):
    """Preprocess data for LSTM model."""
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM Model
def build_model(input_shape):
    """Build and compile the LSTM model."""
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict Price
def predict_price(symbol, sequence_length=50):
    """Predict price using LSTM model."""
    df_15m = fetch_historical_data(symbol, '15m', limit=500)
    df_4h = fetch_historical_data(symbol, '4h', limit=500)

    df_15m = add_technical_indicators(df_15m)
    df_4h = add_technical_indicators(df_4h)

    X_15m, _, scaler_15m = preprocess_data(df_15m, sequence_length)
    X_4h, _, scaler_4h = preprocess_data(df_4h, sequence_length)

    model_15m = build_model((X_15m.shape[1], X_15m.shape[2]))
    model_4h = build_model((X_4h.shape[1], X_4h.shape[2]))

    model_15m.fit(X_15m, X_15m[:, -1, 0], epochs=5, batch_size=32, verbose=1)
    model_4h.fit(X_4h, X_4h[:, -1, 0], epochs=5, batch_size=32, verbose=1)

    latest_sequence_15m = X_15m[-1].reshape(1, X_15m.shape[1], X_15m.shape[2])
    latest_sequence_4h = X_4h[-1].reshape(1, X_4h.shape[1], X_4h.shape[2])

    predicted_price_15m = scaler_15m.inverse_transform([[model_15m.predict(latest_sequence_15m)[0][0], 0, 0, 0, 0, 0]])[0][0]
    predicted_price_4h = scaler_4h.inverse_transform([[model_4h.predict(latest_sequence_4h)[0][0], 0, 0, 0, 0, 0]])[0][0]

    return predicted_price_15m, predicted_price_4h

# Main Function
def analyze_asset(symbol):
    """Analyze an asset using order flow, technical indicators, and LSTM predictions."""
    try:
        # Fetch data
        bids, asks = fetch_order_book(symbol, depth=100)
        trade_data = fetch_trade_data(symbol)
        df_15m = fetch_historical_data(symbol, '15m', limit=500)
        df_4h = fetch_historical_data(symbol, '4h', limit=500)

        if trade_data.empty or df_15m.empty or df_4h.empty:
            logging.warning(f"No data found for {symbol}. Skipping...")
            return

        # Add technical indicators
        df_15m = add_technical_indicators(df_15m)
        df_4h = add_technical_indicators(df_4h)

        # Classify aggressive orders
        trade_data = classify_aggressive_orders(trade_data, bids, asks)

        # Analyze aggressive orders
        aggressive_orders_4h = analyze_aggressive_orders(trade_data, '4h')
        aggressive_orders_15m = analyze_aggressive_orders(trade_data, '15min')

        # Calculate order book depth
        bid_depth, ask_depth = calculate_order_book_depth(bids, asks, depth=100)

        # Identify liquidity zones
        bid_liquidity_zones, ask_liquidity_zones = identify_liquidity_zones(bids, asks, volume_threshold=100, price_tolerance=0.001)

        # Predict prices
        predicted_price_15m, predicted_price_4h = predict_price(symbol)

        # Generate trading signal
        current_price = df_15m['close'].iloc[-1]
        if predicted_price_15m > current_price * 1.002:  # 0.2% threshold
            signal = "🚀 BUY Signal"
        elif predicted_price_15m < current_price * 0.998:  # 0.2% threshold
            signal = "🔻 SELL Signal"
        else:
            signal = "❌ NO TRADE"

        # Output results
        print(f"\n🔹 {symbol}")
        print(f"📈 Current Price: {current_price:.2f}")
        print(f"📊 Trading Signal: {signal}")
        print(f"📊 15M Prediction: {predicted_price_15m:.2f}")
        print(f"📊 4H Prediction: {predicted_price_4h:.2f}")
        print(f"📊 4H Aggressive Buying/Selling: {aggressive_orders_4h['delta'].iloc[-1]:.2f}")
        print(f"📊 15M Aggressive Buying/Selling: {aggressive_orders_15m['delta'].iloc[-1]:.2f}")
        print(f"📊 Order Book Depth (Top 100 Bids/Asks): {bid_depth:.2f} / {ask_depth:.2f}")
        print(f"📊 Bid Liquidity Zones: {[f'{price:.4f} ({vol:.2f})' for price, vol in bid_liquidity_zones]}")
        print(f"📊 Ask Liquidity Zones: {[f'{price:.4f} ({vol:.2f})' for price, vol in ask_liquidity_zones]}")
        print(f"📊 RSI: {df_15m['RSI'].iloc[-1]:.2f}")
        print(f"📊 ATR: {df_15m['ATR'].iloc[-1]:.2f}")
        print(f"📊 OBV: {df_15m['OBV'].iloc[-1]:.2f}")
    except Exception as e:
        logging.error(f"\n🔹 {symbol}")
        logging.error(f"⚠️ Error: {e}")

# List of Assets to Analyze
assets = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT', 'FET/USDT', 'NEAR/USDT', 'AVAX/USDT', 'DOGE/USDT',
          'XRP/USDT', 'LINK/USDT', 'INJ/USDT', 'HNT/USDT', 'CRV/USDT', 'SOL/USDT', 'LTC/USDT',
          'MNT/USDT', 'AAVE/USDT', 'JUP/USDT']

# Analyze each asset
for asset in assets:
    analyze_asset(asset)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 0.0682
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0166
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0097
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0065
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0053
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.1187
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0282
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0177
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0141
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0109
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step

🔹 BTC/USDT
📈 Current Price: 96268.00
📊 Trading Signal: ❌ NO TRADE
📊 15M Prediction: 96317.87
📊 4H Prediction: 97271.07
📊 4H Aggressive Buying/Selling: nan
📊 15M Aggressive Buying/Selling: nan
📊 Order Book Depth (Top 100 Bids/Asks): 12.85 / 12.77
📊 Bid Liquidity Zones: []

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 0.1114
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0184
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0107
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0067
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0048
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 0.1825
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0149
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0070
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0048
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0042
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step

🔹 ETH/USDT
📈 Current Price: 2647.24
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 2666.50
📊 4H Prediction: 2663.75
📊 4H Aggressive Buying/Selling: nan
📊 15M Aggressive Buying/Selling: nan
📊 Order Book Depth (Top 100 Bids/Asks): 297.65 / 567.97
📊 Bid Liquidity Zones: []
📊 Ask Li

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - loss: 0.1674
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0161
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0067
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0051
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0035
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.2626
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0142
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0113
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0094
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0085
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step

🔹 MNT/USDT
📈 Current Price: 1.03
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 1.04
📊 4H Prediction: 1.10
📊 4H Aggressive Buying/Selling: nan
📊 15M Aggressive Buying/Selling: nan
📊 Order Book Depth (Top 100 Bids/Asks): 218815.44 / 221981.90
📊 Bid Liquidity Zones: ['1.0227 (1088

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 0.0597
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0173
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0060
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0027
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0022
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - loss: 0.1024
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.0201
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 0.0127
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0102
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0071
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step

🔹 AAVE/USDT
📈 Current Price: 242.59
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 244.15
📊 4H Prediction: 258.34
📊 4H Aggressive Buying/Selling: nan
📊 15M Aggressive Buying/Selling: nan
📊 Order Book Depth (Top 100 Bids/Asks): 407.81 / 592.36
📊 Bid Liquidity Zones: []
📊 Ask Liqu

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 0.0815
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0096
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0041
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 0.0028
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0021
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - loss: 0.1040
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0178
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0110
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0089
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0064
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step

🔹 JUP/USDT
📈 Current Price: 0.82
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 0.83
📊 4H Prediction: 0.83
📊 4H Aggressive Buying/Selling: 3582.37
📊 15M Aggressive Buying/Selling: 3582.37
📊 Order Book Depth (Top 100 Bids/Asks): 198702.79 / 186889.00
📊 Bid Liquidity Zones: ['0.82

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
import logging
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize KuCoin Exchange
exchange = ccxt.kucoin()

# Fetch Historical Data (OHLCV)
def fetch_historical_data(symbol, timeframe, limit=500):
    """Fetch OHLCV data from KuCoin."""
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df
    except Exception as e:
        logging.error(f"Error fetching historical data for {symbol}: {e}")
        return pd.DataFrame()

# Add Technical Indicators
def add_technical_indicators(df):
    """Add RSI, ATR, OBV, SMA, and EMA to the DataFrame."""
    df['RSI'] = df.ta.rsi(close='close', length=14)
    df['ATR'] = df.ta.atr(high='high', low='low', close='close', length=14)
    df['OBV'] = df.ta.obv(close='close', volume='volume')
    df['SMA_50'] = df.ta.sma(close='close', length=50)
    df['EMA_50'] = df.ta.ema(close='close', length=50)
    return df

# Fetch Level 2 Order Book
def fetch_order_book(symbol, depth=100):
    """Fetch bid-ask Level 2 order book data from KuCoin."""
    try:
        order_book = exchange.fetch_order_book(symbol, limit=depth)
        bids = order_book['bids'][:depth]  # Top bids
        asks = order_book['asks'][:depth]  # Top asks
        return bids, asks
    except Exception as e:
        logging.error(f"Error fetching order book for {symbol}: {e}")
        return [], []

# Fetch Executed Trade Data
def fetch_trade_data(symbol, since=None, limit=1000):
    """Fetch executed market orders from KuCoin."""
    try:
        trades = exchange.fetch_trades(symbol, since=since, limit=limit)
        df = pd.DataFrame(trades, columns=['timestamp', 'price', 'amount', 'side'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        logging.error(f"Error fetching trade data for {symbol}: {e}")
        return pd.DataFrame()

# Classify Active and Passive Liquidity
def classify_liquidity(trade_df, bids, asks, tolerance=0.001):
    """Classify trades as active liquidity (market orders) and passive liquidity (limit orders)."""
    if trade_df.empty or not bids or not asks:
        return trade_df

    best_bid = bids[0][0] if bids else None
    best_ask = asks[0][0] if asks else None

    if not best_bid or not best_ask:
        return trade_df

    # Classify active liquidity (market orders)
    trade_df['active_side'] = trade_df.apply(
        lambda row: 'buy' if abs(row['price'] - best_ask) <= best_ask * tolerance else (
            'sell' if abs(row['price'] - best_bid) <= best_bid * tolerance else None
        ),
        axis=1
    )

    # Weight active liquidity by trade size
    trade_df['weighted_amount'] = trade_df['amount'] * trade_df['price']
    return trade_df

# Analyze Active Liquidity
def analyze_active_liquidity(trade_df, timeframe):
    """Analyze active liquidity (aggressive buy/sell) over a given timeframe."""
    try:
        if trade_df.empty:
            raise ValueError("Trade data is empty.")

        trade_df = trade_df.set_index('timestamp')
        resampled = trade_df.resample(timeframe).agg({
            'weighted_amount': 'sum',  # Weighted by trade size
            'active_side': lambda x: (x == 'buy').sum()
        })

        resampled.rename(columns={'active_side': 'active_buy_count'}, inplace=True)
        resampled['active_sell_count'] = len(trade_df) - resampled['active_buy_count']

        resampled['active_buy_volume'] = trade_df[trade_df['active_side'] == 'buy'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()
        resampled['active_sell_volume'] = trade_df[trade_df['active_side'] == 'sell'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()

        resampled['active_liquidity_ratio'] = (resampled['active_buy_volume'] - resampled['active_sell_volume']) / (
            resampled['active_buy_volume'] + resampled['active_sell_volume'] + 1e-9
        )  # Add small epsilon to avoid division by zero

        return resampled
    except Exception as e:
        logging.error(f"Error analyzing active liquidity for {timeframe}: {e}")
        return pd.DataFrame()

# Calculate Passive Liquidity Ratio
def calculate_passive_liquidity_ratio(bids, asks, current_price, std_dev):
    """Calculate passive liquidity ratio (bid/ask) within ±1 standard deviation."""
    price_range_low = current_price - std_dev
    price_range_high = current_price + std_dev

    # Filter bids and asks within the price range
    bid_volumes = [bid[1] for bid in bids if price_range_low <= bid[0] <= price_range_high]
    ask_volumes = [ask[1] for ask in asks if price_range_low <= ask[0] <= price_range_high]

    total_bid_volume = sum(bid_volumes)
    total_ask_volume = sum(ask_volumes)

    if total_bid_volume + total_ask_volume == 0:
        return 0  # Avoid division by zero

    passive_liquidity_ratio = (total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)
    return passive_liquidity_ratio

# Calculate Order Book Imbalance
def calculate_order_book_imbalance(bids, asks, current_price, std_dev):
    """Calculate order book imbalance within ±1 standard deviation."""
    price_range_low = current_price - std_dev
    price_range_high = current_price + std_dev

    # Filter bids and asks within the price range
    bid_volumes = [bid[1] for bid in bids if price_range_low <= bid[0] <= price_range_high]
    ask_volumes = [ask[1] for ask in asks if price_range_low <= ask[0] <= price_range_high]

    total_bid_volume = sum(bid_volumes)
    total_ask_volume = sum(ask_volumes)

    if total_bid_volume + total_ask_volume == 0:
        return 0  # Avoid division by zero

    order_book_imbalance = (total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)
    return order_book_imbalance

# Preprocess Data for LSTM
def preprocess_data(df, sequence_length=50):
    """Preprocess data for LSTM model."""
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM Model
def build_model(input_shape):
    """Build and compile the LSTM model."""
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict Price
def predict_price(symbol, sequence_length=50):
    """Predict price using LSTM model."""
    df_15m = fetch_historical_data(symbol, '15m', limit=500)
    df_4h = fetch_historical_data(symbol, '4h', limit=500)

    df_15m = add_technical_indicators(df_15m)
    df_4h = add_technical_indicators(df_4h)

    X_15m, _, scaler_15m = preprocess_data(df_15m, sequence_length)
    X_4h, _, scaler_4h = preprocess_data(df_4h, sequence_length)

    model_15m = build_model((X_15m.shape[1], X_15m.shape[2]))
    model_4h = build_model((X_4h.shape[1], X_4h.shape[2]))

    model_15m.fit(X_15m, X_15m[:, -1, 0], epochs=5, batch_size=32, verbose=1)
    model_4h.fit(X_4h, X_4h[:, -1, 0], epochs=5, batch_size=32, verbose=1)

    latest_sequence_15m = X_15m[-1].reshape(1, X_15m.shape[1], X_15m.shape[2])
    latest_sequence_4h = X_4h[-1].reshape(1, X_4h.shape[1], X_4h.shape[2])

    predicted_price_15m = scaler_15m.inverse_transform([[model_15m.predict(latest_sequence_15m)[0][0], 0, 0, 0, 0, 0]])[0][0]
    predicted_price_4h = scaler_4h.inverse_transform([[model_4h.predict(latest_sequence_4h)[0][0], 0, 0, 0, 0, 0]])[0][0]

    return predicted_price_15m, predicted_price_4h

# Main Function
def analyze_asset(symbol):
    """Analyze an asset using order flow, technical indicators, and LSTM predictions."""
    try:
        # Fetch data
        bids, asks = fetch_order_book(symbol, depth=100)
        trade_data = fetch_trade_data(symbol)
        df_15m = fetch_historical_data(symbol, '15m', limit=500)
        df_4h = fetch_historical_data(symbol, '4h', limit=500)

        if trade_data.empty or df_15m.empty or df_4h.empty:
            logging.warning(f"No data found for {symbol}. Skipping...")
            return

        # Add technical indicators
        df_15m = add_technical_indicators(df_15m)
        df_4h = add_technical_indicators(df_4h)

        # Classify active and passive liquidity
        trade_data = classify_liquidity(trade_data, bids, asks)

        # Analyze active liquidity
        active_liquidity_4h = analyze_active_liquidity(trade_data, '4h')
        active_liquidity_15m = analyze_active_liquidity(trade_data, '15min')

        # Calculate dynamic standard deviation
        std_dev = df_15m['close'].std()  # Standard deviation of 1-hour prices
        current_price = df_15m['close'].iloc[-1]

        # Calculate passive liquidity ratio
        passive_liquidity_ratio = calculate_passive_liquidity_ratio(bids, asks, current_price, std_dev)

        # Calculate order book imbalance
        order_book_imbalance = calculate_order_book_imbalance(bids, asks, current_price, std_dev)
         # Identify liquidity zones
        bid_liquidity_zones, ask_liquidity_zones = identify_liquidity_zones(bids, asks, volume_threshold=100, price_tolerance=0.001)

        # Predict prices
        predicted_price_15m, predicted_price_4h = predict_price(symbol)

        # Generate trading signal
        if predicted_price_15m > current_price * 1.002:  # 0.2% threshold
            signal = "🚀 BUY Signal"
        elif predicted_price_15m < current_price * 0.998:  # 0.2% threshold
            signal = "🔻 SELL Signal"
        else:
            signal = "❌ NO TRADE"

        # Output results
        print(f"\n🔹 {symbol}")
        print(f"📈 Current Price: {current_price:.2f}")
        print(f"📊 Trading Signal: {signal}")
        print(f"📊 15M Prediction: {predicted_price_15m:.2f}")
        print(f"📊 4H Prediction: {predicted_price_4h:.2f}")
        print(f"📊 4H Active Liquidity Ratio: {active_liquidity_4h['active_liquidity_ratio'].iloc[-1]:.2f}")
        print(f"📊 15M Active Liquidity Ratio: {active_liquidity_15m['active_liquidity_ratio'].iloc[-1]:.2f}")
        print(f"📊 Passive Liquidity Ratio: {passive_liquidity_ratio:.2f}")
        print(f"📊 Order Book Imbalance: {order_book_imbalance:.2f}")
        print(f"📊 Bid Liquidity Zones: {[f'{price:.4f} ({vol:.2f})' for price, vol in bid_liquidity_zones]}")
        print(f"📊 Ask Liquidity Zones: {[f'{price:.4f} ({vol:.2f})' for price, vol in ask_liquidity_zones]}")
        print(f"📊 RSI: {df_15m['RSI'].iloc[-1]:.2f}")
        print(f"📊 ATR: {df_15m['ATR'].iloc[-1]:.2f}")
        print(f"📊 OBV: {df_15m['OBV'].iloc[-1]:.2f}")
    except Exception as e:
        logging.error(f"\n🔹 {symbol}")
        logging.error(f"⚠️ Error: {e}")

# List of Assets to Analyze
assets = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT', 'FET/USDT', 'NEAR/USDT', 'AVAX/USDT', 'DOGE/USDT',
          'XRP/USDT', 'LINK/USDT', 'INJ/USDT', 'HNT/USDT', 'CRV/USDT', 'SOL/USDT', 'LTC/USDT',
          'MNT/USDT', 'AAVE/USDT', 'JUP/USDT']

# Analyze each asset
for asset in assets:
    analyze_asset(asset)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 0.0484
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0139
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0088
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0081
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0046
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.0656
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0223
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0163
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0113
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0101
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step

🔹 BTC/USDT
📈 Current Price: 96096.00
📊 Trading Signal: ❌ NO TRADE
📊 15M Prediction: 96077.29
📊 4H Prediction: 97456.25
📊 4H Active Liquidity Ratio: nan
📊 15M Active Liquidity Ratio: nan
📊 Passive Liquidity Ratio: 0.28
📊 Order Book Imbalance: 0.28
📊 Bid Liquidity Zones: []
📊 Ask Li

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 0.1264
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0143
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0092
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0064
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0040
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - loss: 0.1298
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0189
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0086
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 0.0071
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0051
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step

🔹 ETH/USDT
📈 Current Price: 2629.45
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 2648.88
📊 4H Prediction: 2702.44
📊 4H Active Liquidity Ratio: nan
📊 15M Active Liquidity Ratio: nan
📊 Passive Liquidity Ratio: 0.40
📊 Order Book Imbalance: 0.40
📊 Bid Liquidity Zones: []
📊 Ask Liq

...

13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.1588
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0130
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0068
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0050
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0035
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 0.3060
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0196
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0149
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0116
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0102
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step

🔹 MNT/USDT
📈 Current Price: 1.03
📊 Trading Signal: ❌ NO TRADE
📊 15M Prediction: 1.04
📊 4H Prediction: 1.12
📊 4H Active Liquidity Ratio: -0.26
📊 15M Active Liquidity Ratio: nan
📊 Passive Liquidity Ratio: -0.01
📊 Order Book Imbalance: -0.01
📊 Bid Liquidity Zones: ['1.0306 (5207.98)'

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0408
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0115
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0047
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0029
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0023
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.3704
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0265
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0201
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0166
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0148
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step

🔹 AAVE/USDT
📈 Current Price: 241.57
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 243.10
📊 4H Prediction: 284.00
📊 4H Active Liquidity Ratio: nan
📊 15M Active Liquidity Ratio: nan
📊 Passive Liquidity Ratio: 0.02
📊 Order Book Imbalance: 0.02
📊 Bid Liquidity Zones: []
📊 Ask Liqui

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 0.0580
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0089
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0032
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0025
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0020
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0523
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0211
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0123
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0077
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0051
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step

🔹 JUP/USDT
📈 Current Price: 0.83
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 0.83
📊 4H Prediction: 0.83
📊 4H Active Liquidity Ratio: nan
📊 15M Active Liquidity Ratio: nan
📊 Passive Liquidity Ratio: 0.18
📊 Order Book Imbalance: 0.18
📊 Bid Liquidity Zones: ['0.8224 (231515.19)'